In [0]:
import dlt
from pyspark.sql.types import StructType, StructField, StringType

# Schema fixo para o JSON de entrada
schema = StructType([
    StructField("appid", StringType(), True),
    StructField("playtime_forever", StringType(), True),
    StructField("playtime_windows_forever", StringType(), True),
    StructField("playtime_mac_forever", StringType(), True),
    StructField("playtime_linux_forever", StringType(), True),
    StructField("playtime_deck_forever", StringType(), True),
    StructField("rtime_last_played", StringType(), True),
    StructField("playtime_disconnected", StringType(), True)
])

@dlt.table(
    name="playtime_b",
    comment="Dados de jogos com deduplicação incremental por appid + playtime_forever"
)
def playtime_ingesta():
    df = (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "json")
            .schema(schema)
            .load("/Volumes/steamdatabricks_workspace/steam/playtime/")
    )
    # Deduplicação por appid + playtime_forever no batch atual
    df = df.dropDuplicates(["appid", "playtime_forever"])
    return df
